<a href="https://colab.research.google.com/github/Adil1979/P6_Tag_Recommendation_Stackoverflow/blob/master/04_APP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Projet6: Catégorisation automatique de questions**

## **1.Librairies**

In [57]:
!pip install gensim # Gensim is an open-source library for unsupervised topic modeling and natural language processing

In [58]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.lines import Line2D
import matplotlib as mpl
#import chart_studio.plotly.plotly as py
from collections import Counter
#import plotly.plotly as py
import seaborn as sns
import datetime as dt
import calendar as cld
from scipy import stats
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import scale
from os import listdir
import glob
import missingno as msno
#from feature_engine.categorical_encoders import OneHotCategoricalEncoder
from sklearn.decomposition import PCA
from matplotlib import dates
import nltk
import re
from nltk.corpus import stopwords 
nltk.download('stopwords')
from nltk.stem.porter import PorterStemmer 
import string
from collections import Counter
from bs4 import BeautifulSoup
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from ast import literal_eval 
from sklearn.externals import joblib
from joblib import load, dump
stop_words = set(stopwords.words('english'))
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.stem.snowball import EnglishStemmer
lemma = WordNetLemmatizer().lemmatize
ps = PorterStemmer()
stemmer = EnglishStemmer()
from joblib import load
random_state = 747

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [59]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [60]:
#nltk.download('all')

In [61]:
set_alpha = set(['a', 'b', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '-'])

## **2.Data**

In [62]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [63]:
model_lr = joblib.load('/content/drive/My Drive/CentraleSupelec_OCR/P6/reg_log_saved.joblib')

In [64]:
dict_tags_500 = joblib.load('/content/drive/My Drive/CentraleSupelec_OCR/P6/list_tag_key_500.joblib')
print(dict_tags_500)

{'redux', 'facebook', 'unicode', 'sql', 'batch-file', 'permissions', 'migration', 'html', 'syntax', 'dns', 'oauth-2.0', 'maven', 'swiftui', 'symfony', 'templates', 'python', 'xamarin', 'sass', 'computer-vision', 'performance', 'swing', 'select', 'kotlin', 'debugging', 'dockerfile', 'jakarta-ee', 'functional-programming', 'vector', 'variables', 'service', 'linq-to-sql', 'compiler-construction', 'shell', 'sharepoint', 'controls', 'jvm', 'asp.net', 'scikit-learn', 'refactoring', 'javafx', 'wcf', 'xsd', 'netbeans', 'unit-testing', 'zend-framework', 'twitter-bootstrap-3', 'gcc', 'matlab', 'hibernate', 'f#', 'pyspark', 'meteor', 'coding-style', 'linux-kernel', 'cocoa-touch', 'kubernetes', 'julia', 'android', '.net-core', 'heroku', 'ruby', 'tomcat', 'c++', 'exception-handling', 'opengl-es', 'python-3.x', 'scroll', 'database-design', 'class', 'c++17', 'parallel-processing', 'timer', 'mongodb', 'webview', 'indexing', 'sql-server', 'import', 'canvas', 'git', 'cakephp', 'c++11', 'flask', 'excepti

In [65]:
type(dict_tags_500)

set

In [66]:
len(dict_tags_500)

500

In [67]:
st_tags_500  = " ".join(dict_tags_500)
st_tags_500

'redux facebook unicode sql batch-file permissions migration html syntax dns oauth-2.0 maven swiftui symfony templates python xamarin sass computer-vision performance swing select kotlin debugging dockerfile jakarta-ee functional-programming vector variables service linq-to-sql compiler-construction shell sharepoint controls jvm asp.net scikit-learn refactoring javafx wcf xsd netbeans unit-testing zend-framework twitter-bootstrap-3 gcc matlab hibernate f# pyspark meteor coding-style linux-kernel cocoa-touch kubernetes julia android .net-core heroku ruby tomcat c++ exception-handling opengl-es python-3.x scroll database-design class c++17 parallel-processing timer mongodb webview indexing sql-server import canvas git cakephp c++11 flask exception oop express jdbc asynchronous collections dom go rust inheritance datetime object iis mysql vba jsp boost file-io asp.net-mvc loops maven-2 css c++14 amazon-s3 jestjs junit ansible browser dependency-injection search image ecmascript-6 node.js 

In [68]:
type(st_tags_500)

str

In [69]:
def word_count(question):
  return len(question.split())

In [70]:
word_count(st_tags_500)

500

In [71]:
question_test = "PHP DOMElement::getElementsByTagName - Anyway to get just the immediate matching children? <p>is there a way to retrieve only the immediate children found by a call to DOMElement::getElementsByTagName? For example, I have an XML document that has a category element. That category element has sub category elements (which have the same structure), like:</p>\n\n<pre><code>&lt;category&gt;\n    &lt;id&gt;1&lt;/id&gt;\n    &lt;name&gt;Top Level Category Name&lt;/name&gt;\n    &lt;subCategory&gt;\n        &lt;id&gt;2&lt;/id&gt;\n        &lt;name&gt;Sub Category Name&lt;/name&gt;\n    &lt;/subCategory&gt;\n    ...\n&lt;/category&gt;\n</code></pre>\n\n<p>If I have a DOMElement representing the top level category, </p>\n\n<pre><code>$topLevelCategoryElement-&gt;getElementsByTagName('id');\n</code></pre>\n\n<p>will return a list with the nodes for all 'id' elements, where I want just the one from the top level. Any way to do this outside of using XPath?</p>\n"
question_test

"PHP DOMElement::getElementsByTagName - Anyway to get just the immediate matching children? <p>is there a way to retrieve only the immediate children found by a call to DOMElement::getElementsByTagName? For example, I have an XML document that has a category element. That category element has sub category elements (which have the same structure), like:</p>\n\n<pre><code>&lt;category&gt;\n    &lt;id&gt;1&lt;/id&gt;\n    &lt;name&gt;Top Level Category Name&lt;/name&gt;\n    &lt;subCategory&gt;\n        &lt;id&gt;2&lt;/id&gt;\n        &lt;name&gt;Sub Category Name&lt;/name&gt;\n    &lt;/subCategory&gt;\n    ...\n&lt;/category&gt;\n</code></pre>\n\n<p>If I have a DOMElement representing the top level category, </p>\n\n<pre><code>$topLevelCategoryElement-&gt;getElementsByTagName('id');\n</code></pre>\n\n<p>will return a list with the nodes for all 'id' elements, where I want just the one from the top level. Any way to do this outside of using XPath?</p>\n"

In [72]:
type(question_test)

str

In [73]:
def preprocessing(text):
  #HTML characters removing
  def remove_html(text):
    html_regex = re.compile('<.*?>') #Compile regular expresions
    return re.sub(html_regex, ' ', str(text)) # Replace regex by ' '
  text0 = remove_html(text)
  

  #URL removing
  def remove_url(text0):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    return re.sub(url_regex, ' ', str(text0))
  text1 = remove_url(text0)
  

  #Punctuation removing
  def remove_punc(text1):
    clean_text = re.sub(r'[?|!|"|:|=|_|{|}|[|]|-|$|%|^|&|]',r' ',str(text1))
    clean_text = re.sub(r'[.|,|)|(|\|/|-|~|`|>|<|*|$|@|;|→]',r' ', clean_text)
    return clean_text
  text2 = remove_url(text1)
  

  def remove_other(text2):
    text = str(text2)
    clean_text = re.sub(r"[^a-zA-Z0-9#+-]", " ", text2.lower())
    return clean_text
  text3 = remove_other(text2)
  

  #Space removing
  def remove_space(text3):
    return ' '.join(str(text3).split())
  text4 = remove_space(text3)
  

  def remove_stopwords(text4):
    text4 = str(text4)
    text4 = " ".join(word for word in text4.split() if word not in gensim.parsing.preprocessing.STOPWORDS and word not in stop_words and word not in set_alpha )
    return text4
  text5 = remove_stopwords(text4)
  

  def tokenize(text5):
    tokens = [lemma(w) for w in text5.split()] #if w.isalpha()
    return tokens
  text6 = tokenize(text5)
  

  def to_string(text6):
    input_string = " ".join(text6)
    return input_string
  text7 = to_string(text6)
  
  def keep_tags(text7):
    text7 = str(text7)
    text7 = " ".join(tag for tag in text7.split() if tag in dict_tags_500)
    return text7
  text8 = keep_tags(text7)
  return text8


In [74]:
question_test_tag = preprocessing(question_test)
print(question_test_tag)

php xml list xpath


In [75]:
question_test_tag_df = pd.DataFrame([question_test_tag], columns=['question'])
question_test_tag_df

,question
0,php xml list xpath


In [76]:
tags_500_df = pd.DataFrame([st_tags_500], columns=['question'])
tags_500_df

,question
0,redux facebook unicode sql batch-file permissi...


In [77]:
test_input_df = pd.concat([question_test_tag_df, tags_500_df], ignore_index=True)
test_input_df

,question
0,php xml list xpath
1,redux facebook unicode sql batch-file permissi...


In [78]:
question_test =  test_input_df['question']
question_test

0                                   php xml list xpath
1    redux facebook unicode sql batch-file permissi...
Name: question, dtype: object

In [79]:
type(question_test)

pandas.core.series.Series

In [80]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [81]:
def tokenize(document):
  tokens = [lemma(w) for w in document.split()]         #if w.isalpha()
  return tokens

In [82]:
stop_words = set(stopwords.words('english'))

In [83]:
vectorizer = TfidfVectorizer(tokenizer = tokenize, stop_words = stop_words, max_features=355)
X_tfidf = vectorizer.fit_transform(question_test).toarray()

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['doe', 'ha', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [84]:
print(X_tfidf.shape)

(2, 355)


In [85]:
feature_names  = vectorizer.get_feature_names()
print(feature_names)

['.htaccess', '.net', '.net-3.5', '.net-core', 'actionscript-3', 'activerecord', 'ajax', 'algorithm', 'amazon-ec2', 'amazon-s3', 'amazon-web-services', 'android', 'android-activity', 'android-emulator', 'android-fragments', 'android-gradle-plugin', 'android-intent', 'android-layout', 'android-studio', 'angular', 'angular-material', 'angular5', 'angularjs', 'animation', 'ansible', 'ant', 'apache', 'apache-kafka', 'apache-spark', 'apache-spark-sql', 'architecture', 'autocomplete', 'awk', 'aws-lambda', 'azure', 'bash', 'batch-file', 'binary', 'binding', 'boost', 'bootstrap-4', 'browser', 'build', 'button', 'c', 'c#', 'c++', 'c++11', 'c++14', 'c++17', 'caching', 'cakephp', 'canvas', 'casting', 'chart', 'checkbox', 'clang', 'class', 'clojure', 'closure', 'cmake', 'cocoa', 'cocoa-touch', 'codeigniter', 'coding-style', 'coldfusion', 'collection', 'color', 'command-line', 'compilation', 'compiler-construction', 'computer-vision', 'concurrency', 'configuration', 'constant', 'constructor', 'cont

In [86]:
len(feature_names)

355

In [87]:
X_test = pd.DataFrame(X_tfidf)
X_test = X_test.iloc[0:1,:]
X_test.columns = feature_names
X_test

,.htaccess,.net,.net-3.5,.net-core,actionscript-3,activerecord,ajax,algorithm,amazon-ec2,amazon-s3,amazon-web-services,android,android-activity,android-emulator,android-fragments,android-gradle-plugin,android-intent,android-layout,android-studio,angular,angular-material,angular5,angularjs,animation,ansible,ant,apache,apache-kafka,apache-spark,apache-spark-sql,architecture,autocomplete,awk,aws-lambda,azure,bash,batch-file,binary,binding,boost,...,vb.net,vba,vector,version-control,video,vim,visual-c++,visual-studio,visual-studio-2008,visual-studio-2010,visual-studio-2012,visual-studio-2015,visual-studio-2017,visual-studio-code,vue.js,vuejs2,wcf,web,web-applications,web-services,webpack,websocket,webview,winapi,window,windows-7,winforms,wix,woocommerce,wordpress,wpf,x86,xamarin,xamarin.forms,xaml,xcode,xml,xpath,xsd,xslt
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.0,0.0


In [88]:
X_test.loc[0,'list'] #c# c# http proxy http proxy dns

0.5

In [89]:
y_pred = model_lr.predict(X_test)
y_pred

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [90]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer(classes=sorted(dict_tags_500))
mlb.fit(dict_tags_500)

MultiLabelBinarizer(classes=['.htaccess', '.net', '.net-3.5', '.net-core',
                             'actionscript-3', 'activerecord', 'ajax',
                             'algorithm', 'amazon-ec2', 'amazon-s3',
                             'amazon-web-services', 'android',
                             'android-activity', 'android-emulator',
                             'android-fragments', 'android-gradle-plugin',
                             'android-intent', 'android-layout',
                             'android-studio', 'angular', 'angular-material',
                             'angular5', 'angularjs', 'animation', 'ansible',
                             'ant', 'apache', 'apache-kafka', 'apache-spark',
                             'apache-spark-sql', ...],
                    sparse_output=False)

In [91]:
tags_text = pd.concat([pd.Series(mlb.inverse_transform(y_pred), name='y_pred')],axis=1)
tags_text = tags_text.to_json()
tags_text
           

'{"y_pred":{"0":["xml"]}}'